# Load Fine-tuned Model for Inference

## Set up and requirements

Clone Conor's ped-detector repo and install requirements

In [10]:
!git clone https://github.com/conorg000/ped-detector.git

Cloning into 'ped-detector'...
remote: Enumerating objects: 2360, done.
remote: Counting objects: 100% (2360/2360), done.
remote: Compressing objects: 100% (1887/1887), done.
remote: Total 2437 (delta 495), reused 2333 (delta 472), pack-reused 77
Receiving objects: 100% (2437/2437), 81.76 MiB | 28.50 MiB/s, done.
Resolving deltas: 100% (528/528), done.
Checking out files: 100% (2290/2290), done.


In [11]:
!pip install -r ped-detector/requirements.txt

ERROR: Could not find a version that satisfies the requirement tarfile (from -r ped-detector/requirements.txt (line 8)) (from versions: none)
ERROR: No matching distribution found for tarfile (from -r ped-detector/requirements.txt (line 8))


Clone the Object Detection API and complete setup

In [3]:
!git clone https://github.com/tensorflow/models.git

Cloning into 'models'...
remote: Enumerating objects: 22, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 34005 (delta 3), reused 2 (delta 0), pack-reused 33983
Receiving objects: 100% (34005/34005), 512.24 MiB | 35.15 MiB/s, done.
Resolving deltas: 100% (21795/21795), done.
Checking out files: 100% (3012/3012), done.


In [0]:
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.

In [5]:
%%bash 
cd models/research
pip install .

Processing /content/models/research
  Created wheel for object-detection: filename=object_detection-0.1-cp36-none-any.whl size=1017522 sha256=6a9d90283be75ec2143f2faef937d56d9096a7f0a81fd0ab899beb036a0383ce
  Stored in directory: /tmp/pip-ephem-wheel-cache-9_gkpsb6/wheels/94/49/4b/39b051683087a22ef7e80ec52152a27249d1a644ccf4e442ea
Successfully built object-detection


In [0]:
%%bash
cp -r /content/models/research/slim/ /usr/local/lib/python3.6/dist-packages/object_detection

In [0]:
%%bash
cd /usr/local/lib/python3.6/dist-packages/object_detection/slim
python setup.py build
python setup.py install

## Inference function

In [12]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
import time
import tensorflow as tf
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util
# patch tf1 into `utils.ops`
utils_ops.tf = tf.compat.v1
# Patch the location of gfile
tf.gfile = tf.io.gfile
"""
Script to load a TF inference graph and perform evaluation on an image file
Returns bounding box coordinates, confidence, inference time
"""

# VARIABLES TO CHANGE
# Change to local env path (wherever you cloned ped-detector)
PATH = '/content/'
# Change to desired jpeg image
image_path = PATH + 'ped-detector/MOT16-01_000001.jpg'

# Path to frozen detection graph. This is the actual fine-tuned model that is used for the object detection.
PATH_TO_CKPT = PATH + 'ped-detector/fine-tuned-model/frozen_inference_graph.pb'
# For pre-trained model
#PATH_TO_CKPT = PATH + 'ped-detector/pre-trained-model/frozen_inference_graph.pb'
# Path to label map
PATH_TO_LABELS = PATH + 'ped-detector/annotations/ped_label_map.pbtxt'
# Detect just one class
NUM_CLASSES = 1

def infer_img(image_path, confidence=0.3):
  """
  Performs pedestrian object detection on an image
  Returns bounding box coordinates (where confidence is over confidence%)
  
  Input:
  image_path (string): path to .jpg file (change paths at top of this script as necessary)
  confidence (float): confidence level (function only returns detections above this level)

  Output:
  results (list of tuples): list containing tuples of detections
                            each tuple is of format (confidence, [ymin, xmin, ymax, xmax])
                            bounding box coordinates are absolute (not relative)
  time (float): inference time in seconds
  """
  # Load the inference graph
  detection_graph = tf.Graph()
  with detection_graph.as_default():
      od_graph_def = tf.GraphDef()
      with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
          serialized_graph = fid.read()
          od_graph_def.ParseFromString(serialized_graph)
          tf.import_graph_def(od_graph_def, name='')

  # Match labels with names and their index
  label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
  categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
  category_index = label_map_util.create_category_index(categories)

  # Resize input image and direct to new path
  img = Image.open(image_path)
  img.thumbnail((600, 600))
  image_path = image_path[:-4] + "_resized.jpg"
  img.save(image_path)

  # Start tf detection graph
  with detection_graph.as_default():
      with tf.Session(graph=detection_graph) as sess:
          # Load tensors
          image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
          detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
          detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
          detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
          num_detections = detection_graph.get_tensor_by_name('num_detections:0')
          # Load image
          image = Image.open(image_path)
          # Make into array
          (im_width, im_height) = image.size
          image_np = np.array(image.getdata()).reshape((im_height, im_width, 3)).astype(np.uint8)
          # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
          image_np_expanded = np.expand_dims(image_np, axis=0)
          # Start inference timer
          t0 = time.time()
          # Get results from inference
          (boxes, scores, classes, num) = sess.run(
              [detection_boxes, detection_scores, detection_classes, num_detections],
              feed_dict={image_tensor: image_np_expanded})
          # Stop inference timer
          t1 = time.time()

  scores = scores.tolist()
  scores = [float(x) for x in scores[0]]
  scores = list(filter(lambda x: x > 0.3, scores))
  num = len(scores)
  inf_time = float(t1-t0)
  bbs = boxes.tolist()[0][:num]
  results = list(zip(scores, bbs))
  for result in results:
    # [ymin, xmin, ymax, xmax]
    result[1][0] *= im_height
    result[1][1] *= im_width
    result[1][2] *= im_height
    result[1][3] *= im_width
  return (results, inf_time)

coordinates, inf_time = infer_img(image_path)
print(coordinates)
print(inf_time)

[(0.9990477561950684, [147.62185633182526, 233.07909965515137, 188.69647920131683, 248.1269359588623]), (0.8932743072509766, [131.4643134176731, 74.06489253044128, 179.7910720705986, 90.08933901786804]), (0.869631290435791, [140.24980255961418, 294.67660188674927, 193.00420653820038, 313.2203221321106]), (0.5049670338630676, [144.36111146211624, 390.92488288879395, 201.4330752491951, 412.1967315673828]), (0.36834245920181274, [131.8284563422203, 508.23705196380615, 184.72096687555313, 529.758095741272]), (0.3588935136795044, [140.45501899719238, 362.12918758392334, 192.76905077695847, 380.97782135009766]), (0.3502677083015442, [138.18467217683792, 311.02402210235596, 201.44870275259018, 333.73517990112305]), (0.3199266195297241, [139.15322586894035, 489.5513892173767, 187.0626618862152, 507.70164728164673])]
0.7000503540039062
